# Select batch-id for execution

In [ ]:
%env DATA_CONNECTOR=alpaca
%env LIU_BROKER=alpaca
batch_id = ## enter batch-id

#### imports

In [ ]:
import asyncio
import json
import math
import sys
from datetime import date, datetime, timedelta

import alpaca_trade_api as tradeapi
import iso8601
import matplotlib.pyplot as plt
import nest_asyncio
import numpy as np
import pandas as pd
import pytz
import requests
from dateutil import parser
from IPython.display import HTML, display, Markdown
from liualgotrader.analytics.analysis import (
    calc_batch_revenue,
    count_trades,
    load_trades_by_batch_id,
    trades_analysis,
    symbol_trade_analytics,
)

from liualgotrader.common import config
from pandas import DataFrame as df
from pytz import timezone
from liualgotrader.common.data_loader import DataLoader 
%matplotlib inline

nest_asyncio.apply()

#### Load batch data

In [ ]:
trades = await load_trades_by_batch_id(batch_id)

In [ ]:
trades

In [ ]:
if trades.empty:
    assert False, "Empty batch. halting execution."

## Calculating batch performance

In [ ]:
how_was_my_batch = trades_analysis(trades, batch_id)

In [ ]:
pd.set_option("display.max_rows", how_was_my_batch.shape[0] + 1)
revenues = round(sum(how_was_my_batch["revenues"]), 2)
traded = len(how_was_my_batch)
win_ratio = round(
        1.0
        * len(how_was_my_batch[how_was_my_batch.revenues >= 0])
        / len(how_was_my_batch[how_was_my_batch.revenues < 0]),
        2,
    ) if len(how_was_my_batch[how_was_my_batch.revenues < 0]) else "high!"
print(
    f"Total revenues=${revenues}\nTotal traded:{traded} Win/Lose ration {win_ratio}"
)

display(
    Markdown(f"{len(how_was_my_batch[how_was_my_batch.revenues >= 0])} **Winners**")
)
display(how_was_my_batch[how_was_my_batch.revenues >= 0].sort_values('revenues'))
display(Markdown(f"{len(how_was_my_batch[how_was_my_batch.revenues < 0])} **Lossers**"))
display(how_was_my_batch[how_was_my_batch.revenues < 0].sort_values('revenues'))

## Display trades in details

In [ ]:
minute_history = DataLoader()
for symbol in trades.symbol.unique():
    symbol_df = trades.loc[trades["symbol"] == symbol]
    start_date = (
        symbol_df["client_time"]
        .min()
        .replace(hour=9, minute=30, second=0, microsecond=0) 
    )
    end_date = (
        symbol_df["client_time"]
        .max()
        .replace(hour=16, minute=0, second=0, microsecond=0)
    )
    symbol_data = minute_history[symbol][start_date:end_date]
    ind = symbol_data.index.get_indexer([start_date, end_date], method="nearest")
    minute_history_index = ind[0]
    end_index = ind[1]
    plt.plot(
        symbol_data.close[minute_history_index:end_index].between_time("9:30", "16:00"),
        label=symbol,
    )
    plt.xticks(rotation=45)

    d, profit = symbol_trade_analytics(symbol_df, plt)

    print(f"{symbol} analysis with profit {round(profit, 2)}")
    display(HTML(pd.DataFrame(data=d).to_html()))
    plt.legend()
    plt.show()